In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tqdm
from tqdm import tqdm
tqdm.pandas()

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn

In [ ]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=dea8bc6ba0fcb14e4bb4151f97092a00bb0c011fcac7091f62daa72d6840044b
  Stored in directory: /root/.cach

In [ ]:
!pip install transformers sentence-transformers scikit-learn

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
from flair.data import Sentence
from flair.nn import Classifier

tagger = Classifier.load("hunflair2")

train_df = pd.read_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_train.csv')
test_df = pd.read_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_test.csv')
val_df = pd.read_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_val.csv')

def extract_entities(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = [(entity.text, entity.tag) for entity in sentence.get_spans('ner')]
    return entities

train_df['entities'] = train_df['text'].progress_apply(extract_entities)
test_df['entities'] = test_df['text'].progress_apply(extract_entities)
val_df['entities'] = val_df['text'].progress_apply(extract_entities)

train_df.to_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_train_entities.csv', index=False)
test_df.to_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_test_entities.csv', index=False)
val_df.to_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_val_entities.csv', index=False)

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_train_entities.csv')
test_df = pd.read_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_test_entities.csv')
val_df = pd.read_csv('/content/drive/My Drive/cohort/mp/admission_only_true/MP_IN_adm_val_entities.csv')

In [ ]:
from collections import defaultdict
import ast

disease_freq = defaultdict(int)
entities_list = train_df['entities'].tolist()

for entities in tqdm(entities_list):
    for entity_text, entity_tag in ast.literal_eval(entities):
        if entity_tag == "Disease":
            disease_freq[entity_text] += 1

sorted_disease_freq = dict(sorted(disease_freq.items(), key=lambda item: item[1], reverse=True))

100%|██████████| 33954/33954 [00:16<00:00, 2093.35it/s]


In [ ]:
len(sorted_disease_freq)

51992

In [ ]:
train_df['diseases_only'] = train_df['entities'].apply(
    lambda entities: [entity[0] for entity in ast.literal_eval(entities) if entity[1] == 'Disease']
)
test_df['diseases_only'] = test_df['entities'].apply(
    lambda entities: [entity[0] for entity in ast.literal_eval(entities) if entity[1] == 'Disease']
)
val_df['diseases_only'] = val_df['entities'].apply(
    lambda entities: [entity[0] for entity in ast.literal_eval(entities) if entity[1] == 'Disease']
)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


model.eval()
model = model.to('cuda')

def generate_embeddings(texts, batch_size=32):
    embeddings = []
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)

    for batch in tqdm(dataloader, desc="Generating embeddings"):
        tokens = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to('cuda')
        with torch.no_grad():
            output = model(**tokens)

        batch_embeddings = output.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.extend(batch_embeddings)

    return embeddings

disease_names = list(disease_freq.keys())

batch_size = 64
embeddings = generate_embeddings(disease_names, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating embeddings:   0%|          | 0/813 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating embeddings: 100%|██████████| 813/813 [00:43<00:00, 18.53it/s]


In [ ]:
import numpy as np

embeddings_filepath = '/content/drive/My Drive/cohort/mp/admission_only_true/disease_embeddings.npy'
np.save(embeddings_filepath, embeddings)

In [ ]:
import faiss
import numpy as np
from tqdm.notebook import tqdm

embeddings_array = np.array(embeddings).astype('float32')
print(len(embeddings_array))

num_clusters = 256

embedding_dim = embeddings_array.shape[1]

res = faiss.StandardGpuResources()
faiss_index = faiss.IndexFlatL2(embedding_dim)
gpu_index = faiss.index_cpu_to_gpu(res, 0, faiss_index)

kmeans = faiss.Clustering(embedding_dim, num_clusters)
kmeans.niter = 200
kmeans.max_points_per_centroid = 50000

kmeans.train(embeddings_array, gpu_index)

centroids = faiss.vector_to_array(kmeans.centroids).reshape(num_clusters, embedding_dim)
_, cluster_ids = gpu_index.search(embeddings_array, 1)
cluster_ids = cluster_ids.flatten()

51992


In [ ]:
word_to_cluster = {disease_names[i]: int(cluster_ids[i]) for i in range(len(disease_names))}

In [ ]:
train_df['cluster_ids'] = train_df['diseases_only'].apply(lambda diseases: [word_to_cluster.get(disease) for disease in diseases])

In [ ]:
def assign_clusters(diseases, word_to_cluster):
    return [word_to_cluster.get(disease, -1) for disease in diseases if disease in word_to_cluster]

test_df['cluster_ids'] = test_df['diseases_only'].apply(lambda diseases: assign_clusters(diseases, word_to_cluster))
val_df['cluster_ids'] = val_df['diseases_only'].apply(lambda diseases: assign_clusters(diseases, word_to_cluster))

In [ ]:
import numpy as np

num_clusters = 256

def create_one_hot_vector(diseases, word_to_cluster, num_clusters):
    one_hot = np.zeros(num_clusters, dtype=int)
    for disease in diseases:
        cluster_id = word_to_cluster.get(disease, -1)
        if cluster_id != -1:
            one_hot[cluster_id] = 1
    return one_hot

train_df['one_hot_features'] = train_df['diseases_only'].apply(
    lambda diseases: create_one_hot_vector(diseases, word_to_cluster, num_clusters)
)

X = np.stack(train_df['one_hot_features'].values)
y = train_df['hospital_expire_flag'].values
print(f"Feature matrix shape: {X.shape}")
print(f"Sample feature vector: {X[0]}")

Feature matrix shape: (33954, 256)
Sample feature vector: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]


In [ ]:
def create_one_hot_vector(cluster_ids, num_clusters):
    one_hot = np.zeros(num_clusters, dtype=int)
    for cluster_id in cluster_ids:
        if cluster_id != -1:
            one_hot[cluster_id] = 1
    return one_hot

num_clusters = 256

test_df['one_hot_features'] = test_df['cluster_ids'].apply(lambda clusters: create_one_hot_vector(clusters, num_clusters))
val_df['one_hot_features'] = val_df['cluster_ids'].apply(lambda clusters: create_one_hot_vector(clusters, num_clusters))

X_test = np.stack(test_df['one_hot_features'].values)
X_val = np.stack(val_df['one_hot_features'].values)
y_test = test_df['hospital_expire_flag'].values
y_val = val_df['hospital_expire_flag'].values

In [ ]:
dtrain = xgb.DMatrix(X, label=y)
dtest = xgb.DMatrix(X_test, label=y_test)
dval = xgb.DMatrix(X_val, label=y_val)

## no resampling

### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X, y)

y_pred_test = svm_classifier.predict(X_test)

print("Test Set:")
print(classification_report(y_test, y_pred_test))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test)}")

y_pred_val = svm_classifier.predict(X_val)

print("\nValidation Set:")
print(classification_report(y_val, y_pred_val))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val)}")

Test Set:
              precision    recall  f1-score   support

           0       0.90      1.00      0.94      8797
           1       0.00      0.00      0.00      1025

    accuracy                           0.90      9822
   macro avg       0.45      0.50      0.47      9822
weighted avg       0.80      0.90      0.85      9822

Accuracy: 0.8956424353492161


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Validation Set:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      4390
           1       0.00      0.00      0.00       518

    accuracy                           0.89      4908
   macro avg       0.45      0.50      0.47      4908
weighted avg       0.80      0.89      0.84      4908

Accuracy: 0.8944580277098615


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "eta": 0.1,
    "max_depth": 200
}

model = xgb.train(
    params, dtrain, num_boost_round=200, evals=[(dtrain, 'train'), (dval, 'eval')], verbose_eval=10
)

y_pred = model.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred_binary))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

[0]	train-logloss:0.33556	eval-logloss:0.34796
[10]	train-logloss:0.22588	eval-logloss:0.32248
[20]	train-logloss:0.16180	eval-logloss:0.31988
[30]	train-logloss:0.12077	eval-logloss:0.32314
[40]	train-logloss:0.09404	eval-logloss:0.32881
[50]	train-logloss:0.07624	eval-logloss:0.33536
[60]	train-logloss:0.06401	eval-logloss:0.34222
[70]	train-logloss:0.05512	eval-logloss:0.34894
[80]	train-logloss:0.04838	eval-logloss:0.35482
[90]	train-logloss:0.04320	eval-logloss:0.36049
[100]	train-logloss:0.03911	eval-logloss:0.36594
[110]	train-logloss:0.03588	eval-logloss:0.37046
[120]	train-logloss:0.03317	eval-logloss:0.37584
[130]	train-logloss:0.03091	eval-logloss:0.38005
[140]	train-logloss:0.02899	eval-logloss:0.38348
[150]	train-logloss:0.02730	eval-logloss:0.38725
[160]	train-logloss:0.02587	eval-logloss:0.39082
[170]	train-logloss:0.02461	eval-logloss:0.39394
[180]	train-logloss:0.02350	eval-logloss:0.39698
[190]	train-logloss:0.02252	eval-logloss:0.40028
[199]	train-logloss:0.02174	eva

## smote

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X, y)

print("Original class distribution:", Counter(y))
print("Class distribution after SMOTE:", Counter(y_train_smote))

Original class distribution: Counter({0: 30420, 1: 3534})
Class distribution after SMOTE: Counter({0: 30420, 1: 30420})


### svm

In [ ]:
svm_classifier_smote = SVC(kernel='linear', C=1.0)
svm_classifier_smote.fit(X_train_smote, y_train_smote)

y_pred_test_smote = svm_classifier_smote.predict(X_test)

print("\nTest Set (SMOTE):")
print(classification_report(y_test, y_pred_test_smote))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test_smote)}")

y_pred_val_smote = svm_classifier_smote.predict(X_val)

print("\nValidation Set (SMOTE):")
print(classification_report(y_val, y_pred_val_smote))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val_smote)}")


Test Set (SMOTE):
              precision    recall  f1-score   support

           0       0.90      0.78      0.84      8797
           1       0.13      0.27      0.17      1025

    accuracy                           0.73      9822
   macro avg       0.51      0.53      0.50      9822
weighted avg       0.82      0.73      0.77      9822

Accuracy: 0.7271431480350234

Validation Set (SMOTE):
              precision    recall  f1-score   support

           0       0.90      0.78      0.84      4390
           1       0.13      0.29      0.18       518

    accuracy                           0.73      4908
   macro avg       0.52      0.53      0.51      4908
weighted avg       0.82      0.73      0.77      4908

Accuracy: 0.7259576202118989


### xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

dtrain = xgb.DMatrix(X_train_smote, label=y_train_smote)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "eta": 0.01,
    "max_depth": 200
}

model = xgb.train(
    params, dtrain, num_boost_round=200, evals=[(dtrain, 'train'), (dval, 'eval')], verbose_eval=10
)

y_pred = model.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred_binary))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

[0]	train-logloss:0.68644	eval-logloss:0.68890
[10]	train-logloss:0.62706	eval-logloss:0.65125
[20]	train-logloss:0.57618	eval-logloss:0.62009
[30]	train-logloss:0.53167	eval-logloss:0.59384
[40]	train-logloss:0.49240	eval-logloss:0.57171
[50]	train-logloss:0.45764	eval-logloss:0.55284
[60]	train-logloss:0.42680	eval-logloss:0.53635
[70]	train-logloss:0.39899	eval-logloss:0.52112
[80]	train-logloss:0.37379	eval-logloss:0.50783
[90]	train-logloss:0.35115	eval-logloss:0.49646
[100]	train-logloss:0.33065	eval-logloss:0.48653
[110]	train-logloss:0.31195	eval-logloss:0.47759
[120]	train-logloss:0.29499	eval-logloss:0.46947
[130]	train-logloss:0.27962	eval-logloss:0.46254
[140]	train-logloss:0.26552	eval-logloss:0.45641
[150]	train-logloss:0.25256	eval-logloss:0.45086
[160]	train-logloss:0.24065	eval-logloss:0.44595
[170]	train-logloss:0.22973	eval-logloss:0.44166
[180]	train-logloss:0.21951	eval-logloss:0.43767
[190]	train-logloss:0.21010	eval-logloss:0.43436
[199]	train-logloss:0.20223	eva

##full oversampling

In [ ]:
import pandas as pd
from sklearn.utils import resample

positive_class = train_df[train_df['hospital_expire_flag'] == 1]
negative_class = train_df[train_df['hospital_expire_flag'] == 0]

positive_oversampled = resample(
    positive_class,
    replace=True,
    n_samples=len(negative_class),
    random_state=42
)

balanced_train_df = pd.concat([negative_class, positive_oversampled])

balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Class distribution after oversampling:")
print(balanced_train_df['hospital_expire_flag'].value_counts())

Class distribution after oversampling:
hospital_expire_flag
1    30420
0    30420
Name: count, dtype: int64


In [ ]:
X_train_balanced = np.stack(balanced_train_df['one_hot_features'].values)
y_train_balanced = balanced_train_df['hospital_expire_flag'].values

### logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_classifier = LogisticRegression(max_iter=1000)
logreg_classifier.fit(X_train_balanced, y_train_balanced)

y_pred_test_logreg = logreg_classifier.predict(X_test)

print("\nTest Set (Logistic Regression - Balanced):")
print(classification_report(y_test, y_pred_test_logreg))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test_logreg)}")

y_pred_val_logreg = logreg_classifier.predict(X_val)

print("\nValidation Set (Logistic Regression - Balanced):")
print(classification_report(y_val, y_pred_val_logreg))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val_logreg)}")


Test Set (Logistic Regression - Balanced):
              precision    recall  f1-score   support

           0       0.94      0.70      0.80      8797
           1       0.20      0.64      0.30      1025

    accuracy                           0.69      9822
   macro avg       0.57      0.67      0.55      9822
weighted avg       0.87      0.69      0.75      9822

Accuracy: 0.6936469150885767

Validation Set (Logistic Regression - Balanced):
              precision    recall  f1-score   support

           0       0.94      0.70      0.80      4390
           1       0.19      0.60      0.29       518

    accuracy                           0.69      4908
   macro avg       0.57      0.65      0.55      4908
weighted avg       0.86      0.69      0.75      4908

Accuracy: 0.6919315403422983


### random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_balanced, y_train_balanced)

y_pred_test_rf = rf_classifier.predict(X_test)

print("\nTest Set (Random Forest - Balanced):")
print(classification_report(y_test, y_pred_test_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test_rf)}")

y_pred_val_rf = rf_classifier.predict(X_val)

print("\nValidation Set (Random Forest - Balanced):")
print(classification_report(y_val, y_pred_val_rf))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val_rf)}")


Test Set (Random Forest - Balanced):
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      8797
           1       0.21      0.03      0.06      1025

    accuracy                           0.89      9822
   macro avg       0.56      0.51      0.50      9822
weighted avg       0.83      0.89      0.85      9822

Accuracy: 0.8863775198533903

Validation Set (Random Forest - Balanced):
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4390
           1       0.20      0.04      0.06       518

    accuracy                           0.88      4908
   macro avg       0.55      0.51      0.50      4908
weighted avg       0.82      0.88      0.85      4908

Accuracy: 0.8832518337408313


### gradient boosted trees


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_classifier = GradientBoostingClassifier(random_state=42)
gb_classifier.fit(X_train_balanced, y_train_balanced)

y_pred_test_gb = gb_classifier.predict(X_test)

print("\nTest Set (Gradient Boosting - Balanced):")
print(classification_report(y_test, y_pred_test_gb))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test_gb)}")

y_pred_val_gb = gb_classifier.predict(X_val)

print("\nValidation Set (Gradient Boosting - Balanced):")
print(classification_report(y_val, y_pred_val_gb))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val_gb)}")


Test Set (Gradient Boosting - Balanced):
              precision    recall  f1-score   support

           0       0.94      0.72      0.82      8797
           1       0.20      0.60      0.30      1025

    accuracy                           0.71      9822
   macro avg       0.57      0.66      0.56      9822
weighted avg       0.86      0.71      0.76      9822

Accuracy: 0.7120749338220321

Validation Set (Gradient Boosting - Balanced):
              precision    recall  f1-score   support

           0       0.94      0.72      0.81      4390
           1       0.20      0.58      0.29       518

    accuracy                           0.71      4908
   macro avg       0.57      0.65      0.55      4908
weighted avg       0.86      0.71      0.76      4908

Accuracy: 0.7059902200488998


### svm

In [ ]:
svm_classifier_resampled = SVC(kernel='linear', C=1.0)
svm_classifier_resampled.fit(X_train_balanced, y_train_balanced)

y_pred_test_resampled = svm_classifier_resampled.predict(X_test)

print("\nTest Set (SMOTE):")
print(classification_report(y_test, y_pred_test_resampled))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test_resampled)}")

y_pred_val_resampled = svm_classifier_resampled.predict(X_val)

print("\nValidation Set (SMOTE):")
print(classification_report(y_val, y_pred_val_resampled))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val_resampled)}")

### xgboost

In [ ]:
xgb_classifier = xgb.XGBClassifier(random_state=42)
xgb_classifier.fit(X_train_balanced, y_train_balanced)

y_pred_test_gb = xgb_classifier.predict(X_test)

print("\nTest Set (xgboost - Balanced):")
print(classification_report(y_test, y_pred_test_gb))
print(f"Accuracy: {accuracy_score(y_test, y_pred_test_gb)}")

y_pred_val_gb = xgb_classifier.predict(X_val)

print("\nValidation Set (xgboost - Balanced):")
print(classification_report(y_val, y_pred_val_gb))
print(f"Accuracy: {accuracy_score(y_val, y_pred_val_gb)}")


Test Set (xgboost - Balanced):
              precision    recall  f1-score   support

           0       0.93      0.81      0.87      8797
           1       0.22      0.46      0.30      1025

    accuracy                           0.77      9822
   macro avg       0.57      0.64      0.58      9822
weighted avg       0.85      0.77      0.81      9822

Accuracy: 0.7747912848706985

Validation Set (xgboost - Balanced):
              precision    recall  f1-score   support

           0       0.92      0.81      0.86      4390
           1       0.21      0.43      0.28       518

    accuracy                           0.77      4908
   macro avg       0.57      0.62      0.57      4908
weighted avg       0.85      0.77      0.80      4908

Accuracy: 0.7703748981255094
